In [1]:
import pandas as pd

import torch
from epam.dnsm import train_model, TransformerBinarySelectionModel
from epam.sequences import translate_sequences


In [2]:
pcp_df = pd.read_csv("~/data/wyatt-10x-1p5m_pcp_2023-10-07.csv")

In [3]:
nhead = 4
dim_feedforward = 2048
layer_count = 3
# model = train_model(pcp_df, nhead=nhead, dim_feedforward=dim_feedforward, layer_count=layer_count, batch_size=32, num_epochs=10, learning_rate=0.001, checkpoint_dir="./_checkpoints", log_dir="./_logs")

In [4]:
model = TransformerBinarySelectionModel(
    nhead=nhead, dim_feedforward=dim_feedforward, layer_count=layer_count
)

model.load_state_dict(torch.load("./_checkpoints/model_epoch_9.pth")["model_state_dict"])

Using Metal Performance Shaders


<All keys matched successfully>

In [5]:
[aa_str] = translate_sequences([pcp_df.loc[0, "parent"]])
model.p_substitution_of_aa_str(aa_str)

array([0.023111  , 0.02445051, 0.02492264, 0.02321235, 0.02583305,
       0.02255295, 0.02380204, 0.02425446, 0.02253114, 0.02318418,
       0.02384661, 0.02309757, 0.02311187, 0.02465206, 0.02149563,
       0.02293609, 0.02423076, 0.02346496, 0.02379945, 0.02333233,
       0.02121226, 0.02562145, 0.02413822, 0.02333023, 0.02250639,
       0.02362912, 0.02291543, 0.02376284, 0.02446552, 0.02333317,
       0.02365008, 0.02322046, 0.02387234, 0.02210846, 0.02393964,
       0.02355381, 0.0255277 , 0.02361007, 0.02246664, 0.02280878,
       0.02479593, 0.02237235, 0.02149357, 0.02423572, 0.02381271,
       0.02273981, 0.02339267, 0.02318927, 0.02223376, 0.02326695,
       0.02432246, 0.02266584, 0.02327654, 0.02298066, 0.02261778,
       0.02320118, 0.02415984, 0.0267837 , 0.02552385, 0.02531306,
       0.02163774, 0.02361843, 0.02280953, 0.02413983, 0.0233677 ,
       0.02372977, 0.02137937, 0.02313062, 0.02485378, 0.02355475,
       0.02366747, 0.02317392, 0.02297806, 0.02378938, 0.02352